# CHILD Knowledge Management
## Seattle Children Hospital
### July 2025
#### Jerome Massot (jeromemassot@google.com)

## 00- Setup and Import Modules

requirements.txt

- beautifulsoup4
- google-cloud-discoveryengine
- google-storage
- google-genai

In [ ]:
%%writefile requirements.txt
html-to-markdown
beautifulsoup4
google-cloud-discoveryengine
google-cloud-storage
google-genai

Overwriting requirements.txt


In [ ]:
! pip install -q -r "/content/requirements.txt"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.3/187.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 26.4 MB/s eta 0:00:00


In [1]:
from google.cloud import discoveryengine_v1beta
from google.cloud import storage
from google import genai

In [2]:
from html_to_markdown import convert_to_markdown
from bs4 import XMLParsedAsHTMLWarning
from bs4 import BeautifulSoup
import requests

In [3]:
from pydantic import BaseModel, Field
from collections import defaultdict
from typing import List

In [4]:
import warnings
import pprint
import tqdm
import json
import uuid
import re

In [5]:
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

## 01- Patient Eduction website

The page education pages can be seen as a collection of PDFs documents and links to internal and external websites. There is almost no informational content in these pages, beyond these links to PDFs and other websites.

The extraction consists in the retrieving of the PDFs documents, and URLs pointing to internal webpages, external webpaces, and videos content.

### 01-01- Pages Extraction

We start by parsing the patient education landing page for extracting all the references to the conditions pages.

An atomic page data contains:
- a title (str)
- an url (str)
- a collection of PDFs (list of uri using the pattern gs://...
- a collection of videos (YouTube)
- a collection of internal links
- a collection of external links
- a collection of data impossible to retrieved

In [ ]:
url = "https://www.seattlechildrens.org/patients-families/patient-education/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
pages = defaultdict(lambda: defaultdict)
for link in soup.find_all("a"):
    if link.get("href").startswith("/clinics/"):
        # take care of the website naming unconsistencies
        is_resources = "resources" in link.get("href").split("/")[-2] or "education" in link.get("href").split("/")[-2]
        if is_resources:
            pages[link.text] = {
                "title": link.text,
                "url": "https://www.seattlechildrens.org" + link.get("href"),
                "pdfs": [],
                "videos": [],
                "internal_links": [],
                "external_links": [],
                "failed_links": []
            }

# manually add the vascular access which does not respect the pattern of other pages
pages["Vascular Access"] = {
    "title": "Vascular Access",
    "url": "https://www.seattlechildrens.org/clinics/vascular-access-service/",
    "pdfs": [],
    "videos": [],
    "internal_links": [],
    "external_links": [],
    "failed_links": []
}

The dictionary contains all the urls for the conditions, the keys are the conditions titles.

In [ ]:
print(f"Number of pages found: {len(pages)}...")
for condition_name, url in pages.items():
    print(condition_name, url)
    break

Number of pages found: 72...
Airway and Esophageal Resources {'title': 'Airway and Esophageal Resources', 'url': 'https://www.seattlechildrens.org/clinics/airway-esophageal-center/patient-and-family-resources/', 'pdfs': [], 'videos': [], 'internal_links': [], 'external_links': [], 'failed_links': []}


### 01-02- Embedded PDFs, Videos, and Hyperlinks extraction

- A PDF object data contains:
  - a title (str)
  - an url (from sch website)
  - an gs location (str using the gs:// pattern)
  - a language (language used in the PDF)

- A Video data contains:
  - an url (YouTube format)

- An Hyperlink data contains:
  - an url

In [ ]:
PDFS_BUCKET_NAME = "sch_patient_education_pdfs"

In [ ]:
def extract_pdfs_videos_hyperlinks(
    condition_name: str, pages: dict, bucket_name: str, debug: bool=False
) -> None:
    """
    Extract all the PDF files, videos urls, and hyperlinks from a webpage.
    :param condition_name: name of the condition
    :param pages: dictionary containing pages information
    :param bucket_name: name of the bucket where the PDFs are stored
    :param debug: debug flag
    :return: None
    """

    # retrieve the condition title and page url
    title = pages[condition_name]["title"]
    url = pages[condition_name]["url"]

    # create the bucket if it does not exist
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    if not bucket.exists():
        bucket.create()

    # extracting the knowledge embedded as links
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # pdfs list to be added to the page information
    pdfs = []
    mapping_for_multilingual_pdfs = {}

    # hyperlinks and video lists to the page
    internal_links = []
    external_links = []
    failed_links = []
    videos = []

    for link in tqdm.tqdm(soup.find_all("a")):

        if debug:
            print(f'Working on {link.get("href")}...')

        # enrich the url if needed
        if not link.get("href"):
            continue
        if link.get("href").startswith("http"):
            url = link.get("href")
        else:
            url = "https://www.seattlechildrens.org" + link.get("href")

        # check if the link points to a PDF file
        if url[-3:].lower() == "pdf":

            # get the pdf name
            file_name = url.split("/")[-1].lower()

            # get the pdf language
            language = link.get("lang", "en")

            # to manage the titles of the documents written in other language than English
            if language == 'en':
                mapping_for_multilingual_pdfs[url.split("/")[-1][:-4].lower()] = link.text
                title = link.text
            else:
                current_pdf = url.split("/")[-1].lower()
                for reference_pdf in mapping_for_multilingual_pdfs.keys():
                    if reference_pdf in current_pdf:
                        title = mapping_for_multilingual_pdfs[reference_pdf]
                        break

            # append the current pdf to the list of page's pdfs
            pdfs.append(
                {
                    "title": title,
                    "url": url,
                    "gs_location": f"gs://{bucket_name}/{condition_name.lower()}/{file_name})",
                    "language": language
                }
            )

            # download the pdf in the gs bucket
            if debug:
                print(f"Downloading {url}...")

            try:
                response = requests.get(url, timeout=10)
                if response.status_code != 200:
                    failed_links.append(url)
                else:
                    pdf_content = response.content
                    folder_name = condition_name.lower().replace(" ", "_").encode("ascii", "ignore").decode("ascii")
                    file_name = file_name.replace(" ", "_").encode("ascii", "ignore").decode("ascii")
                    blob = bucket.blob(f"{folder_name}/{file_name}")
                    blob.upload_from_string(pdf_content, content_type="application/pdf")
            except:
                failed_links.append(url)
        else:
            # append the url to the list of external links or videos
            if url.startswith("https://www.youtube.com"):
                  videos.append(url)
            if url.startswith("https://www.seattlechildrens.org"):
                internal_links.append(url)
            else:
                external_links.append(url)

    # update the pages dictionary
    pages[condition_name]["pdfs"] = pdfs
    pages[condition_name]["internal_links"] = internal_links
    pages[condition_name]["external_links"] = external_links
    pages[condition_name]["failed_links"] = failed_links

    # videos list to be added to the page information
    for link in soup.find_all("a", class_="video-link"):
        videos.append(link.get("href"))
    pages[condition_name]["videos"] = videos

    print(f"Extracted {len(pdfs)} PDFs for condition {condition_name}...")
    print(f"Extracted {len(videos)} videos for condition {condition_name}...")
    print(f"Extracted {len(internal_links)} internal links for condition {condition_name}...")
    print(f"Extracted {len(external_links)} external links for condition {condition_name}...")
    print(f"Failed for {len(failed_links)} links for condition {condition_name}...")

In [ ]:
extract_pdfs_videos_hyperlinks("Craniofacial Resources", pages, PDFS_BUCKET_NAME, False)

100%|██████████| 224/224 [00:19<00:00, 11.33it/s]

Extracted 32 PDFs for condition Craniofacial Resources...
Extracted 2 videos for condition Craniofacial Resources...
Extracted 135 internal links for condition Craniofacial Resources...
Extracted 56 external links for condition Craniofacial Resources...
Failed for 0 links for condition Craniofacial Resources...


In [ ]:
for condition_name in pages.keys():
    print(f"Working on {condition_name}...")
    extract_pdfs_videos_hyperlinks(condition_name, pages, PDFS_BUCKET_NAME)

Working on Airway and Esophageal Resources...


100%|██████████| 169/169 [00:14<00:00, 11.85it/s]


Extracted 51 PDFs for condition Airway and Esophageal Resources...
Extracted 1 videos for condition Airway and Esophageal Resources...
Extracted 100 internal links for condition Airway and Esophageal Resources...
Extracted 18 external links for condition Airway and Esophageal Resources...
Failed for 0 links for condition Airway and Esophageal Resources...
Working on Anesthesiology Resources...


100%|██████████| 161/161 [00:28<00:00,  5.56it/s]


Extracted 28 PDFs for condition Anesthesiology Resources...
Extracted 3 videos for condition Anesthesiology Resources...
Extracted 108 internal links for condition Anesthesiology Resources...
Extracted 25 external links for condition Anesthesiology Resources...
Failed for 0 links for condition Anesthesiology Resources...
Working on Apheresis Resources...


...

Extracted 238 PDFs for condition Urology Resources...
Extracted 2 videos for condition Urology Resources...
Extracted 110 internal links for condition Urology Resources...
Extracted 38 external links for condition Urology Resources...
Failed for 0 links for condition Urology Resources...
Working on Vascular Anomalies Resources...


100%|██████████| 168/168 [00:25<00:00,  6.72it/s]


Extracted 24 PDFs for condition Vascular Anomalies Resources...
Extracted 1 videos for condition Vascular Anomalies Resources...
Extracted 114 internal links for condition Vascular Anomalies Resources...
Extracted 30 external links for condition Vascular Anomalies Resources...
Failed for 0 links for condition Vascular Anomalies Resources...
Working on Vascular Access...


100%|██████████| 240/240 [02:25<00:00,  1.65it/s]

Extracted 117 PDFs for condition Vascular Access...
Extracted 17 videos for condition Vascular Access...
Extracted 89 internal links for condition Vascular Access...
Extracted 33 external links for condition Vascular Access...
Failed for 0 links for condition Vascular Access...


In [ ]:
print(f"{len(pages)} pages have been parsed from the Patient Education website ...")

72 pages have been parsed from the Patient Education website ...


In [ ]:
for artifact_type in ['pdfs', 'videos', 'internal_links', 'external_links']:
    count = sum([len(condition[artifact_type]) for condition in pages.values()])
    print(f"{count} {artifact_type} have been extracted from the Patient Education website ...")

4397 pdfs have been extracted from the Patient Education website ...
267 videos have been extracted from the Patient Education website ...
7829 internal_links have been extracted from the Patient Education website ...
2741 external_links have been extracted from the Patient Education website ...


Ok so now let's save the pages dictionary as a JSON file.

In [ ]:
with open("pages.json", "w") as f:
    json.dump(pages, f)

We can also duplicate it in the GSC bucket used for the PDF extractions.

In [ ]:
# upload the pages.json to GSC bucket
!gsutil cp patient_education_pages.json gs://{PDFS_BUCKET_NAME}/patient_education_pages.json

Copying file://pages.json [Content-Type=application/json]...
/ [1 files][  1.7 MiB/  1.7 MiB]                                                
Operation completed over 1 objects/1.7 MiB.                                      


## 02- Conditions website

The conditions webpages contain a lot of information available as text content. The number of embedded PDF documnents is low.

- The knowledge extraction is focus on the text content. The text content is organized in chunks created from the pages hierarchy.

- The external and internal hyperlinks are also referenced for each page.

### 02-01- Pages Extraction

This section of the notebook extracts the list of conditions pages from the index page available in the Seattle Children website.

In [ ]:
def extract_conditions_collection(url: str) -> dict:
    """
    Extract the collection of conditions pages
    :param url: url of the index page
    :return: a dictionary with the conditions names and urls
    """

    # prefix for the links
    prefix = "https://www.seattlechildrens.org"

    # get the source of the page
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    soup = BeautifulSoup(response.content, 'html.parser')

    # list the links corresponding to the conditions pages
    conditions_pages = {
        a.text: prefix+a.get('href') for a in soup.find_all('a', href=True) if a.get('href').startswith("/conditions")
    }

    return conditions_pages

In [ ]:
conditions_pages = extract_conditions_collection("https://www.seattlechildrens.org/conditions/a-z/")

In [ ]:
print(f"There are {len(conditions_pages)} Condition pages in the collection...")

There are 392 Condition pages in the collection...


### 02-02- Content Extraction and Chunking

The schema for the page object is slightly different as the one used for the Patient Education pages, because there are more information to be extracted from the Conditions pages and also to include the references to the chunks.

In [ ]:
def chunk_markdown(markdown_content: str, links: dict) -> list:
    """
    Chunk the markdown content
    :param markdown_content: content in markdown format
    :param links: dictionary containing the links extracted from content
    :return: list of chunks
    """

    # list of chunks
    chunks = []

    # chunking is based on the mardown layout
    lines = markdown_content.split("\n")

    # current chunk
    current_id = -1
    def default_chunk():
        return {
            "id": current_id + 1,
            "unique_id": str(uuid.uuid4()),
            "content": "",
            "parent_id": None,
            "url_links": [],
            "is_root": False,
            "kind": "chunk"
        }

    # init the different artifacts
    current_chunk = default_chunk()
    hierarchy_ids = dict()

    # fill the chunks line by line
    for i, line in enumerate(lines):
        if len(line)>0:

            # Markdown layour drives the chunks content
            hashtag_count = line[:10].count("#")

            if hashtag_count == 0:
                current_chunk['content'] += "\n" + line

                # gather the found links
                for link in links:
                    if link in line:
                        current_chunk['url_links'].append(links[link])
            else:
                current_chunk['url_links'] = list(set(current_chunk['url_links']))
                chunks.append(current_chunk)
                current_chunk = default_chunk()
                current_chunk['content'] += line + "\n"
                hierarchy_ids[hashtag_count] = current_chunk['id']

                if hashtag_count == 1:
                    current_chunk['is_root'] = True
                else:
                    if hashtag_count == 2:
                        current_chunk['parent_id'] = 0
                    elif hashtag_count > 2:
                        if hashtag_count-1 in hierarchy_ids:
                            current_chunk['parent_id'] = hierarchy_ids[hashtag_count-1]
                        else:
                            current_chunk['parent_id'] = hierarchy_ids[min(hierarchy_ids.keys())]

                # gather the found links
                for link in links:
                    if link in line:
                        current_chunk['url_links'].append(links[link])

                # incrementing the id counter for next chunk
                current_id = current_chunk['id']
    chunks.append(current_chunk)
    return chunks[1:]

In [ ]:
def extract_from_page(url: str) -> dict:
    """
    Extract knowledge from Conditions Page.
    :param url: url of the condition page
    :return: content and metadata as dictionary
    """

    # prefix for the links
    prefix = "https://www.seattlechildrens.org"

    # get the source of the page
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    soup = BeautifulSoup(response.content, 'html.parser')

    # extract the title of the page
    title = soup.find('div', class_="mod page-title")
    if title:
        title = title.find('h1').text
        title_content = f"# {title}"
    else:
        title = "Title Not Found"

    # treate the accordeons
    div_elements = soup.find_all('div', class_='accordion-header heading4 js-accordion-header')
    for div_tag in div_elements:
        new_h3_tag = soup.new_tag('h3')
        new_h3_tag.extend(div_tag.contents)
        div_tag.replace_with(new_h3_tag)
    div_elements = soup.find_all('div', class_='accordion-more js-accordion-more')
    for div_tag in div_elements:
        div_tag.unwrap()
    uv_elements = soup.find_all('ul', class_='accordion accordion--classic js-accordion')
    for uv_element in uv_elements:
        uv_element.unwrap()
    li_elements = soup.find_all('li', class_='accordion-item js-accordion-item')
    for li_element in li_elements:
        li_element.unwrap()

    # extract the main content of the page
    main_content = soup.find('div', class_="main-content-body")

    if main_content:

        markdown_content = convert_to_markdown(
            main_content,
            heading_style='atx',
            escape_asterisks=True
        )

        link_regex = re.compile(r'(!?\[[(^)\]]*\])\((.*?)\)')
        link_matches = link_regex.findall(markdown_content)
        if link_matches:
            for link_match in link_matches:
                target = link_match[1]
                if target.startswith("http"):
                    prefixed_target = target
                else:
                    prefixed_target = prefix + target
                pattern = r"\(" + target.replace("/", "\\/") + "\\)"
                markdown_content = re.sub(pattern, f"({prefixed_target})", markdown_content)
    else:
        markdown_content = "Main content not found"

    # copyrights
    copyright = soup.find('p', class_="ho-psmall")
    if copyright:
        copyright = copyright.text
    else:
        copyright = "Copyright not found"

    # extract links
    def clean_link(link_value: str):
        link_value = link_value.replace("\r\n", "").strip()
        return link_value

    # Links parsing for page and chunks
    links = []
    internal_links = []
    external_links = []
    if main_content:
        internal_links = [prefix+a.get('href') for a in main_content.find_all('a', href=True) if not a.get('href').startswith("http")]
        external_links = [a.get('href') for a in main_content.find_all('a', href=True) if a.get('href').startswith("http")]
        links = {
            clean_link(a.text): prefix+a.get('href') for a in main_content.find_all('a', href=True)
        }

    # create the page content dictionary
    page_content = {
        "unique_id": str(uuid.uuid4()),
        "title": title,
        "content": title_content + markdown_content,
        "copyright": copyright,
        "chunks": chunk_markdown(title_content + markdown_content, links),
        "kind": "page",
        "external_links": external_links,
        "internal_links": internal_links
    }

    return page_content

In [ ]:
test = extract_from_page(conditions_pages['22q11.2-Related Disorders'])

In [ ]:
pprint.pprint(test['chunks'])

[{'content': '# 22q11.2-Related Disorders## What are 22q11\\.2\\-related '
             'disorders?\n'
             '\n'
             '22q11\\.2\\-related disorders are caused by differences in part '
             'of chromosome 22, called the q11\\.2 region. Chromosomes contain '
             'genes, which tell our cells how to work and what proteins to '
             'make. There are 23 pairs of chromosomes in each cell of the '
             'body.\n'
             '22q11\\.2\\-related disorders happen in at least 1 in 1,000 '
             'newborns.\n'
             'The symptoms differ widely, even among members of the same '
             'family. There may be small differences in how your child’s '
             'eyelids, nose and ears look.\n'
             'These conditions are linked to many health issues. They can '
             'affect your child’s growth, feeding, breathing, speaking, '
             'hearing, learning and mental health. But most children with '
             '22q

In [ ]:
with open("one_condition.jsonl", "w") as fp:
    json.dump(test, fp)

Let's extract the content of all the conditions pages available in the collection.

In [ ]:
pages_content = []
pages_w_issue = []
for condition, page_url in tqdm.tqdm(conditions_pages.items()):
    if condition != "Conditions" and condition != "\r\n        All Symptoms\r\n    ":
        try:
            pages_content.append(extract_from_page(page_url))
        except:
            pages_w_issue.append(condition)

100%|██████████| 392/392 [11:15<00:00,  1.72s/it]


In [ ]:
print(f"{len(pages_content)} Condition pages extracted...")

389 Condition pages extracted...


In [ ]:
pages_w_issue

['Mental Health Problems']

Let's save the page content dict as a JSON file.

In [ ]:
jsonl_content = ""
for page_content in pages_content:
    if "unique_id" not in page_content.keys():
        print(page_content)
    jsonl_content += json.dumps(page_content) + "\n"


with open("pages_content.jsonl", "w") as fp:
    fp.write(jsonl_content)

In [ ]:
# upload the pages.json to GSC bucket
! gsutil cp pages_content.jsonl gs://sch_conditions_pages/pages_content.jsonl

Copying file://pages_content.jsonl [Content-Type=application/octet-stream]...
/ [1 files][ 10.3 MiB/ 10.3 MiB]                                                
Operation completed over 1 objects/10.3 MiB.                                     


Let's summarize the amount of information extracted.

In [ ]:
for page_content in pages_content:
    print(page_content)
    break

{'unique_id': 'bde8c4d6-693a-46f9-8cc1-b204eaf34d07', 'title': '22q11.2-Related Disorders', 'content': '# 22q11.2-Related Disorders## What are 22q11\\.2\\-related disorders?\n\n22q11\\.2\\-related disorders are caused by differences in part of chromosome 22, called the q11\\.2 region. Chromosomes contain genes, which tell our cells how to work and what proteins to make. There are 23 pairs of chromosomes in each cell of the body.\n\n22q11\\.2\\-related disorders happen in at least 1 in 1,000 newborns.\n\nThe symptoms differ widely, even among members of the same family. There may be small differences in how your child’s eyelids, nose and ears look.\n\nThese conditions are linked to many health issues. They can affect your child’s growth, feeding, breathing, speaking, hearing, learning and mental health. But most children with 22q11\\.2\\-related disorders only have problems in some of these areas.\n\n### What causes 22q11\\.2\\-related disorders?\n\nThese disorders happen because of cha

In [ ]:
print(f"{len(pages_content)} Condition pages analyzed...")
for artifact_type in ['chunks', 'internal_links', 'external_links']:
    count = sum([len(page_content[artifact_type]) for page_content in pages_content])
    print(f"{count} {artifact_type} have been extracted from the Conditions website ...")

389 Condition pages analyzed...
8281 chunks have been extracted from the Conditions website ...
7207 internal_links have been extracted from the Conditions website ...
390 external_links have been extracted from the Conditions website ...


## 03- Pathways website

The pathways website contains only PDF documents which are extracted using the following code.

In [10]:
url = "https://www.seattlechildrens.org/healthcare-professionals/community-providers/pathways/"

In [11]:
PDFS_BUCKET_NAME = "sch_pathways_pdfs"

In [21]:
def extract_pdfs(url: str, bucket_name: str, debug: bool=False) -> None:
    """
    Extract all the PDF files
    :param url: url of the root webpace
    :param bucket_name: name of the bucket where the PDFs are stored
    :param debug: debug flag
    :return: None
    """

    # create the bucket if it does not exist
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    if not bucket.exists():
        bucket.create()

    # extracting the knowledge embedded as links
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # placeholder for the failed PDF urls
    failed_links = []

    # extracted PDFs counter
    success_count = 0

    for link in tqdm.tqdm(soup.find_all("a")):

        # enrich the url if needed
        if not link.get("href"):
            continue

        url = "https://www.seattlechildrens.org" + link.get("href")

        # check if the link points to a PDF file
        if url[-3:].lower() == "pdf":

            if debug:
              print(f'Working on {link.get("href")}...')
              print(f'URL: {url}')

            try:
                response = requests.get(url, timeout=10)
                if debug:
                    print(f'Status code: {response.status_code}')
                    return
                if response.status_code != 200:
                    failed_links.append(url)
                else:
                    pdf_content = response.content
                    file_name = url.split("/")[-1].lower()
                    file_name = file_name.replace(" ", "_").encode("ascii", "ignore").decode("ascii")
                    blob = bucket.blob(file_name)
                    blob.upload_from_string(pdf_content, content_type="application/pdf")
                    success_count += 1
            except:
                failed_links.append(url)

    print(f"Extracted {success_count} PDFs from pathways ...")
    print(f"Failed to extract {len(failed_links)} PDFs from pathways ...")

In [22]:
extract_pdfs(url, PDFS_BUCKET_NAME)

100%|██████████| 201/201 [02:08<00:00,  1.57it/s]

Extracted 87 PDFs from pathways ...
Failed to extract 0 PDFs from pathways ...


## 04- Emergency and Urgent Care

The Emergency or Urgent Care page contains information about conditions and the most appropriate service to contact for each of them.

The main source of knowledge is embedded in a table that needs to be parsed consistently.

In [23]:
url = "https://www.seattlechildrens.org/clinics/urgent-care-clinic/emergency-or-urgent-care"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

### 04-01- Table Extraction

In [24]:
# prefix for the links
prefix = "https://www.seattlechildrens.org"

In [62]:
def extract_table(url: str)-> dict:
    """
    Extract table content from url
    :param url: url of the table
    :return: dictionary containing the table content
    """

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    conditions_services = {}

    # table extraction
    table_content = soup.find("table")
    tbody = table_content.find("tbody")

    for tr in tbody.find_all("tr"):
        tds = list(tr.find_all("td"))

        # condition parsing
        condition_td = tds[0]
        if condition_td.find("a"):
            # removing the information for the em anchor
            if condition_td.find("em"):
                condition_td.find("em").decompose()
                condition_url = None
            else:
                condition_url = prefix + condition_td.find("a").get("href")
            condition_name = condition_td.text.strip()
            # removing all non ascii characters from condition_name
            condition_name = condition_name.encode("ascii", "ignore").decode("ascii")
        else:
            condition_name = condition_td.text.strip()
            condition_url = None
        condition = {
            "name": condition_name,
            "url": condition_url
        }

        # service parsing
        service_td = tds[1]
        if service_td.find("a"):
            service_name = service_td.find("a").text.strip()
            service_url = prefix + service_td.find("a").get("href")
        else:
            service_name = condition_td.text.strip()
            service_url = None
        service = {
            "name": service_name,
            "url": service_url
        }

        conditions_services[condition_name] = {
            "condition": condition,
            "service": service
        }

    print(f"{len(conditions_services)} rows have been extracted ...")
    return conditions_services

In [63]:
conditions_services = extract_table(url)

39 rows have been extracted ...


In [68]:
with open("emergency_or_urgent_care.json", "w") as fp:
    json.dump(conditions_services, fp)

In [69]:
! gsutil cp emergency_or_urgent_care.json gs://sch_emergency_or_urgent_care_pdfs/emergency_or_urgent_care.json

Copying file://emergency_or_urgent_care.json [Content-Type=application/json]...
/ [1 files][  8.5 KiB/  8.5 KiB]                                                
Operation completed over 1 objects/8.5 KiB.                                      


#### 04-02- PDFs extraction

In [65]:
PDFS_BUCKET_NAME = "sch_emergency_or_urgent_care_pdfs"

In [66]:
extract_pdfs(url, PDFS_BUCKET_NAME)

100%|██████████| 227/227 [00:13<00:00, 17.19it/s]

Extracted 16 PDFs from pathways ...
Failed to extract 0 PDFs from pathways ...


#### 04-03- Extraction of the text content

In [ ]:
# TO DO: Keep only the content in the  <div class="main-content-body">
# TO DO: decompose <ul class="block-featured-buttons" data-block-type="FeaturedButtonBlockData" data-block-name="Buttons" data-block-id="85609">
# TO DO: manage the accordion li id="" class="accordion-item js-accordion-item">